In [ ]:
%pip install --upgrade langchain langchain-community langchainhub langchain-openai chromadb unstructured[all-docs] ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate

In [ ]:
from dotenv import load_dotenv
load_dotenv()

False

In [ ]:
# Load, chunk and index the contents of the blog.
loader = UnstructuredFileLoader("/content/drive/Shareddrives/SJSU_Data298/data/quoraQA.txt")

docs = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(openai_api_key='XXX'))

In [ ]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
template = '''You are a chatbot that communicates like a math professor. Use the following pieces of retrieved-context to answer the question. If you don't know the answer, just answer like elon musk would. Use three sentences maximum and keep the answer concise.
Context: {context}
Question: {question}
Answer: '''
prompt = ChatPromptTemplate.from_template(template = template)
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key='XXX')


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:

def parse_questions(file_path):
  with open(file_path) as fp:
    for line in fp.readlines():
      yield line.strip()

In [ ]:
answers = []
for question in parse_questions("/content/drive/Shareddrives/SJSU_Data298/data/QuoraQ.txt"):
  answer = rag_chain.invoke(question)
  answers.append(dict(question = question, answers = answers))
  print(question,answer)

Question: Does defining the real numbers via Dedekind cuts imply the least upper bound property of the reals, or does it require you to assume in advance that the real numbers have the l. u. b. property? Or neither?, Defining the real numbers via Dedekind cuts implies the least upper bound property of the reals. This construction shows that the real numbers have the least upper bound property. It does not require assuming it in advance.
Question: As a person who, by default, uses radians, I've seen people say that degrees in 180 are composite. In my opinion, that doesn't make sense when some people think 5040 is the best composite number out there. Why still the ancient 180?, The choice of 180 degrees as a measure of angles is deeply rooted in historical and cultural practices. The base 60 system used by ancient civilizations like the Sumerians and Babylonians influenced the division of angles into 360 degrees. The concept of 180 degrees being composite is a result of this historical c

In [ ]:
list(parse_questions("/content/drive/Shareddrives/SJSU_Data298/data/QuoraQ.txt"))



['Question: Does defining the real numbers via Dedekind cuts imply the least upper bound property of the reals, or does it require you to assume in advance that the real numbers have the l. u. b. property? Or neither?,',
 "Question: As a person who, by default, uses radians, I've seen people say that degrees in 180 are composite. In my opinion, that doesn't make sense when some people think 5040 is the best composite number out there. Why still the ancient 180?,",
 'Question: How might AI contribute to solving complex global challenges such as climate change, poverty, and healthcare disparities?',
 'Question: Is there a map projection which is conformal and is equal-area?',
 'Question: Why does zero equal itself when divided by a different number?',
 "Question: Why doesn't any mathematician call himself an algebraist?",
 'Question: Were Pythagoras and Archimedes Albanians or modern Greeks ?',
 'Question: What is the origin of the term "crazy cat lady"? What is the meaning of the term "